In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, recall_score, precision_score, accuracy_score, f1_score
from itertools import combinations
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv("../WA_Fn-UseC_-HR-Employee-Attrition.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
monthlyIncomeBins = [0,2000,4000,6000,8000,10000,20000]
monthlyIncomeLabels = [0,1,2,3,4,5]
df['MonthlyIncomeBins'] = pd.cut(df['MonthlyIncome'], bins=monthlyIncomeBins, labels=monthlyIncomeLabels)
df = df.dropna()
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'MonthlyIncomeBins'],
      dtype='object')

In [3]:
# Adding new variables
df['Time_in_each_comp'] = (df['Age']-20) / ((df)['NumCompaniesWorked']+1)
df['TotalSatisfaction_mean'] = (df['RelationshipSatisfaction']  + df['EnvironmentSatisfaction'] + df['JobSatisfaction'] + df['JobInvolvement'] + df['WorkLifeBalance'])/5
df['Income_YearsComp'] = df['MonthlyIncome'] / df['YearsAtCompany']
df['Income_YearsComp'] = df['Income_YearsComp'].replace(np.Inf, 0)
df['Income_Distance'] = df['MonthlyIncome'] / df['DistanceFromHome']
df['Fidelity'] = (df['NumCompaniesWorked']) / df['TotalWorkingYears']
df['Fidelity'] = df['Fidelity'].replace(np.Inf, 0)
df['Stability'] = df['YearsInCurrentRole'] / df['YearsAtCompany']
df['Stability'].fillna((df['Stability'].mean()), inplace=True)
df['Hrate_Mrate'] = df['HourlyRate'] / df['MonthlyRate']
def SalesDpt(df) :
    if df['Department'] == 'Sales':
        return 1
    else:
        return 0
df['SalesDpt'] = df.apply(lambda df:SalesDpt(df) ,axis = 1)
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'MonthlyIncomeBins', 'Time_in_each_comp',
       'TotalSatisfaction_mean', 'Income_YearsComp', 'Income_Distance',
       'Fidelity', 'Stability', 'Hrate_Mrate', 'SalesDpt'],
      dtype='object')

In [4]:
#'Age','Attrition','DistanceFromHome','EnvironmentSatisfaction','Gender','HourlyRate','JobLevel','JobSatisfaction','MaritalStatus',
#'MonthlyIncome','NumCompaniesWorked','PercentSalaryHike','PerformanceRating','RelationshipSatisfaction','StockOptionLevel',
#'TotalWorkingYears','TrainingTimesLastYear','WorkLifeBalance','YearsAtCompany','YearsInCurrentRole','YearsSinceLastPromotion','YearsWithCurrManager'

features=['Gender','JobLevel','JobSatisfaction','MaritalStatus',
'MonthlyIncome','NumCompaniesWorked','PercentSalaryHike','PerformanceRating','RelationshipSatisfaction','StockOptionLevel',
'TotalWorkingYears','TrainingTimesLastYear','WorkLifeBalance','YearsAtCompany','YearsInCurrentRole','YearsSinceLastPromotion','YearsWithCurrManager']

features=['Department','JobRole','Gender', 'StockOptionLevel','WorkLifeBalance',
          'Education','JobInvolvement','JobLevel','YearsAtCompany','DistanceFromHome',
          'JobSatisfaction','NumCompaniesWorked','PerformanceRating','MonthlyIncome',
          'YearsInCurrentRole','MaritalStatus']

features=['DistanceFromHome', 'NumCompaniesWorked', 'StockOptionLevel', 'MonthlyIncomeBins', 'YearsAtCompany', 'MaritalStatus', 'YearsInCurrentRole', 'Education', 'Department', 'JobInvolvement']

features = ['OverTime', 'Time_in_each_comp', 'TotalSatisfaction_mean', 'Income_YearsComp', 'Income_Distance',
           'Fidelity', 'DailyRate', 'Stability', 'YearsWithCurrManager', 'StockOptionLevel',
           'Hrate_Mrate', 'HourlyRate', 'MonthlyRate', 'TotalWorkingYears', 'SalesDpt']

features

['OverTime',
 'Time_in_each_comp',
 'TotalSatisfaction_mean',
 'Income_YearsComp',
 'Income_Distance',
 'Fidelity',
 'DailyRate',
 'Stability',
 'YearsWithCurrManager',
 'StockOptionLevel',
 'Hrate_Mrate',
 'HourlyRate',
 'MonthlyRate',
 'TotalWorkingYears',
 'SalesDpt']

In [5]:
#Select features
data=df[features]
data_binary_encoded = pd.get_dummies(data)
X=data_binary_encoded
y=df["Attrition"].replace({'Yes': 1, 'No': 0})
X

,Time_in_each_comp,TotalSatisfaction_mean,Income_YearsComp,Income_Distance,Fidelity,DailyRate,Stability,YearsWithCurrManager,StockOptionLevel,Hrate_Mrate,HourlyRate,MonthlyRate,TotalWorkingYears,SalesDpt,OverTime_No,OverTime_Yes
0,2.333333,2.2,998.833333,5993.000000,1.000000,1102,0.666667,5,0,0.004826,94,19479,8,1,0,1
1,14.500000,2.8,513.000000,641.250000,0.100000,279,0.700000,7,1,0.002449,61,24907,10,0,1,0
2,2.428571,2.8,0.000000,1045.000000,0.857143,1373,0.596061,0,0,0.038397,92,2396,7,0,0,1
3,6.500000,3.2,363.625000,969.666667,0.125000,1392,0.875000,0,0,0.002418,56,23159,8,0,0,1
4,0.700000,2.6,1734.000000,1734.000000,1.500000,591,1.000000,2,1,0.002405,40,16632,6,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,3.200000,3.4,514.200000,111.782609,0.235294,884,0.400000,3,1,0.003336,41,12290,17,0,1,0
1466,3.800000,2.2,1427.285714,1665.166667,0.444444,613,1.000000,7,1,0.001957,42,21457,9,0,1,0
1467,3.500000,2.6,1023.666667,1535.500000,0.166667,155,0.333333,3,1,0.016815,87,5174,6,0,0,1
1468,9.666667,2.8,598.888889,2695.000000,0.117647,1023,0.666667,8,0,0.004757,63,13243,17,1,1,0


In [6]:
#Function to create Model
def createModel(X):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    X_scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    model = LogisticRegression()
    model = model.fit(X_train_scaled, y_train)
    trainScore = model.score(X_train_scaled,y_train)
    testScore = model.score(X_test_scaled,y_test)
    test = model.predict(X)
    precisionScore = precision_score(y,test)
    accuracyScore = accuracy_score(y,test)
    f1Score = f1_score(y,test)
    recallScore = recall_score(y,test)
    classRep = classification_report(y,test)
    retList = [trainScore, testScore, precisionScore, accuracyScore, f1Score, recallScore, classRep]
    return retList

createModel(X)

[0.8756805807622504,
 0.8505434782608695,
 0.18639455782312925,
 0.5251700680272109,
 0.28189300411522633,
 0.5780590717299579,
 '              precision    recall  f1-score   support\n\n           0       0.86      0.52      0.65      1233\n           1       0.19      0.58      0.28       237\n\n    accuracy                           0.53      1470\n   macro avg       0.53      0.55      0.46      1470\nweighted avg       0.75      0.53      0.59      1470\n']

In [ ]:
def createModel(X):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    X_scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    model = LogisticRegression()
    model = model.fit(X_train_scaled, y_train)
    trainScore = model.score(X_train_scaled,y_train)
    testScore = model.score(X_test_scaled,y_test)
    test = model.predict(X)
    precisionScore = precision_score(y,test, labels=np.unique(y_pred))
    accuracyScore = accuracy_score(y,test, labels=np.unique(y_pred))
    f1Score = f1_score(y,test, labels=np.unique(y_pred))
    recallScore = recall_score(y,test, labels=np.unique(y_pred))
    classRep = classification_report(y,test, labels=np.unique(y_pred))
    retList = [trainScore, testScore, precisionScore, accuracyScore, f1Score, recallScore, classRep]
    return retList

createModel(X)

In [7]:
# This will create different combinations of features and will iterate through the newModel function. 
minCombinations = 7
newFeatures = []
for i in range(minCombinations, len(features)):
    newFeatures += list(combinations(features, i))

# Uncomment the next line to create a fast example
#newFeatures = list(combinations(features, len(features)-1))
scores = []
counter = 1

for i in newFeatures:
    print(f'Progress {round(counter/len(newFeatures)*100,1)}% {counter}/{len(newFeatures)}')
    counter += 1
    newDF = df.filter(i, axis=1)
    X = pd.get_dummies(newDF)
    retList = createModel(X)
    retList.append(i)
    scores.append(retList)

print("Models processing complete!")

Progress 0.1% 1/1940


/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

Progress 0.1% 2/1940
Progress 0.2% 3/1940
Progress 0.2% 4/1940


/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

Progress 0.3% 5/1940


...

Progress 98.1% 1904/1940
Progress 98.2% 1905/1940
Progress 98.2% 1906/1940
Progress 98.3% 1907/1940
Progress 98.4% 1908/1940
Progress 98.4% 1909/1940
Progress 98.5% 1910/1940
Progress 98.5% 1911/1940
Progress 98.6% 1912/1940
Progress 98.6% 1913/1940
Progress 98.7% 1914/1940
Progress 98.7% 1915/1940
Progress 98.8% 1916/1940
Progress 98.8% 1917/1940
Progress 98.9% 1918/1940


/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

Progress 98.9% 1919/1940
Progress 99.0% 1920/1940
Progress 99.0% 1921/1940
Progress 99.1% 1922/1940
Progress 99.1% 1923/1940
Progress 99.2% 1924/1940
Progress 99.2% 1925/1940
Progress 99.3% 1926/1940
Progress 99.3% 1927/1940
Progress 99.4% 1928/1940
Progress 99.4% 1929/1940
Progress 99.5% 1930/1940


/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

Progress 99.5% 1931/1940
Progress 99.6% 1932/1940
Progress 99.6% 1933/1940
Progress 99.7% 1934/1940
Progress 99.7% 1935/1940
Progress 99.8% 1936/1940
Progress 99.8% 1937/1940
Progress 99.9% 1938/1940
Progress 99.9% 1939/1940
Progress 100.0% 1940/1940
Models processing complete!


In [8]:
# Converting to Dataframe and Sorting
scoresDF = pd.DataFrame(scores)
scoresDF.columns = ["trainScore", "testScore", "precisionScore","accuracyScore", "f1Score","recallScore", "classificationReport", "Features"]
scoresDF = scoresDF.sort_values(["f1Score"], ascending=False)
scoresDF

,trainScore,testScore,precisionScore,accuracyScore,f1Score,recallScore,classificationReport,Features
549,0.866606,0.823370,0.210456,0.544898,0.319430,0.662447,precision recall f1-score ...,"(OverTime, Time_in_each_comp, Income_YearsComp..."
1055,0.856624,0.831522,0.206186,0.528571,0.315893,0.675105,precision recall f1-score ...,"(Time_in_each_comp, TotalSatisfaction_mean, In..."
1052,0.852995,0.831522,0.218241,0.603401,0.314924,0.565401,precision recall f1-score ...,"(Time_in_each_comp, TotalSatisfaction_mean, In..."
386,0.873866,0.831522,0.196635,0.446259,0.314815,0.789030,precision recall f1-score ...,"(OverTime, Time_in_each_comp, TotalSatisfactio..."
1792,0.861162,0.834239,0.196410,0.447619,0.314189,0.784810,precision recall f1-score ...,"(Time_in_each_comp, TotalSatisfaction_mean, In..."
...,...,...,...,...,...,...,...,...
330,0.870236,0.834239,0.000000,0.838776,0.000000,0.000000,precision recall f1-score ...,"(OverTime, Time_in_each_comp, TotalSatisfactio..."
1268,0.850272,0.823370,0.000000,0.838776,0.000000,0.000000,precision recall f1-score ...,"(Time_in_each_comp, Income_YearsComp, Fidelity..."
1278,0.850272,0.823370,0.000000,0.838776,0.000000,0.000000,precision recall f1-score ...,"(Time_in_each_comp, Income_Distance, Fidelity,..."
324,0.866606,0.850543,0.000000,0.838776,0.000000,0.000000,precision recall f1-score ...,"(OverTime, Time_in_each_comp, TotalSatisfactio..."


In [9]:
# Uncomment the following line to export dataframe to csv file
scoresDF.to_csv('scoresDF2.csv', index = False, header=True)

In [10]:
# Example of printing out a Classification Report
print(scoresDF.loc[7][7])

('OverTime', 'Time_in_each_comp', 'TotalSatisfaction_mean', 'Income_YearsComp', 'Income_Distance', 'Fidelity', 'DailyRate', 'Stability', 'YearsWithCurrManager', 'Hrate_Mrate', 'TotalWorkingYears')


In [11]:
print(scoresDF.loc[1][7])

('OverTime', 'Time_in_each_comp', 'TotalSatisfaction_mean', 'Income_YearsComp', 'Income_Distance', 'Fidelity', 'DailyRate', 'Stability', 'YearsWithCurrManager', 'StockOptionLevel', 'HourlyRate')


In [20]:
# Used to calculate amount of combinations
testCombinations = []
for i in range(6, len(features)):
    testCombinations += list(combinations(features, i))

len(testCombinations)

27823

In [17]:
filterDF = pd.read_csv("scoresDF2.csv")
filterDF = filterDF.loc[scoresDF["f1Score"]>.25].loc[scoresDF["precisionScore"]>.2].loc[scoresDF["accuracyScore"]>.2].loc[scoresDF["recallScore"]>.3]
countDict = {} 
for feature in features:
    countDict[feature] = len(filterDF[filterDF['Features'].str.contains(feature)])

sortedDict = sorted(countDict.items(), key=lambda x: x[1], reverse=True)
for i in sortedDict:
    print(i[0], i[1])

Income_YearsComp 130
TotalWorkingYears 122
StockOptionLevel 120
OverTime 119
SalesDpt 119
DailyRate 118
YearsWithCurrManager 118
HourlyRate 117
Time_in_each_comp 116
TotalSatisfaction_mean 114
Stability 114
MonthlyRate 114
Fidelity 113
Hrate_Mrate 111
Income_Distance 93


In [18]:
filterDF

,trainScore,testScore,precisionScore,accuracyScore,f1Score,recallScore,classificationReport,Features
36,0.861162,0.842391,0.196759,0.482313,0.308810,0.717300,precision recall f1-score ...,"('Time_in_each_comp', 'TotalSatisfaction_mean'..."
43,0.871143,0.842391,0.196952,0.487075,0.308257,0.708861,precision recall f1-score ...,"('OverTime', 'TotalSatisfaction_mean', 'Income..."
55,0.855717,0.823370,0.191824,0.438776,0.307305,0.772152,precision recall f1-score ...,"('OverTime', 'Time_in_each_comp', 'Income_Year..."
59,0.867514,0.826087,0.193059,0.453741,0.307161,0.751055,precision recall f1-score ...,"('OverTime', 'Time_in_each_comp', 'Income_Year..."
109,0.869328,0.817935,0.204578,0.557823,0.305556,0.603376,precision recall f1-score ...,"('OverTime', 'Income_Distance', 'Fidelity', 'D..."
...,...,...,...,...,...,...,...,...
1890,0.861162,0.845109,0.000000,0.838776,0.000000,0.000000,precision recall f1-score ...,"('OverTime', 'Time_in_each_comp', 'TotalSatisf..."
1893,0.869328,0.836957,0.000000,0.838776,0.000000,0.000000,precision recall f1-score ...,"('OverTime', 'Time_in_each_comp', 'TotalSatisf..."
1915,0.851180,0.823370,0.000000,0.838776,0.000000,0.000000,precision recall f1-score ...,"('Income_YearsComp', 'Income_Distance', 'Fidel..."
1918,0.862069,0.836957,0.000000,0.838776,0.000000,0.000000,precision recall f1-score ...,"('TotalSatisfaction_mean', 'Income_YearsComp',..."
